In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/dummy.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,title,text,event_type,arguments,date,metadata
0,3109,3109,"Five Iraqi Shia, Sunni and Kurdish political p...",The accord was the second step towards rebuild...,Sign Agreement,"[{'start': 15, 'end': 73, 'type': 'Agreement C...",August 2007,['(BBC)']
1,20427,7826,Millions in Yemen ‘a step away from starvation’,"The crisis in Yemen, now in its seventh year o...",Famine,"[{'start': 14, 'end': 18, 'type': 'Areas affec...",NaN,NaN
2,2769,2769,"Police forcibly break up a protest in Malé, Ma...",Maldives police have used pepper spray to brea...,Protest_Online Condemnation,"[{'start': 101, 'end': 111, 'type': 'Arrested'...",April 2016,['(BBC)']
3,20133,7532,Pakistan marble mine collapse kills 22; dozens...,Army rescuers take part in a rescue search for...,Mine Collapses,"[{'start': 76, 'end': 118, 'type': 'Location',...",NaN,NaN
4,1749,1749,"Near the town of Ramle, Israeli archeologists ...",Hundreds of whisky and gin bottles found at re...,New archeological discoveries,"[{'start': 12, 'end': 33, 'type': 'Historical ...",March 2017,['(The Guardian)']


In [3]:
from src.data import DoceeForInference

dataset = DoceeForInference(df, use_title=True)

In [4]:
from transformers.utils import PaddingStrategy
from transformers import AutoTokenizer, pipeline

pretrained_model_name_or_path = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=pretrained_model_name_or_path,
    padding=PaddingStrategy.MAX_LENGTH,
    use_fast=True
)
summarizer = pipeline(
    "summarization",
    model=pretrained_model_name_or_path,
    tokenizer=tokenizer,
    device=0,
    framework="pt"
)


In [5]:
eg = dataset[0]

In [6]:
min_length = 20
max_length = 200
num_beams = 10
batch_size = 1
num_workers = 1

output = summarizer(
    eg,
    min_length=min_length,
    max_length=max_length,
    num_beams=num_beams,
    truncation=True,
    batch_size=batch_size,
    num_workers=num_workers
)
output

[{'summary_text': ' Five Iraqi Shia, Sunni and Kurdish political parties sign a deal forming a new "Moderates Front" Nouri al–Maliki says the accord is the second step towards rebuilding Iraq\'s political process . The White House hails the deal as "an important symbol" of unity in Iraq .'}]

In [7]:
def create_summarizer(
        df,
        use_title,
        pretrained_model_name_or_path="sshleifer/distilbart-cnn-12-6",
        min_length=20,
        max_length=200,
        num_beams=10
):
    tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        padding=PaddingStrategy.MAX_LENGTH,
        use_fast=True
    )
    summarizer = pipeline(
        "summarization",
        model=pretrained_model_name_or_path,
        tokenizer=tokenizer,
        device=0,
        framework="pt"
    )
    dataset = DoceeForInference(df, use_title=use_title)
    eg = dataset[0]

    def get():
        return summarizer(
            eg,
            min_length=min_length,
            max_length=max_length,
            num_beams=num_beams,
            truncation=True,
            batch_size=1,
            num_workers=1
        )

    return get


In [8]:
get_no_title = create_summarizer(
    df,
    use_title=False,
)

In [9]:
several_summaries = set(get_no_title()[0]["summary_text"] for _ in range(5))
always_equal = len(several_summaries) == 1
always_equal


True

In [10]:
def check_consistency(get_summary):
    bunch = set(get_summary() for _ in range(5))
    return len(bunch) == 1

summary_extractor = lambda do_inference: lambda: do_inference()[0]["summary_text"]

In [11]:
get_with_title = create_summarizer(
    df,
    use_title=True
)
check_consistency(summary_extractor(get_with_title))

True